In [1]:
import os
import warnings

warnings.simplefilter(action="ignore")
os.environ["GRPC_VERBOSITY"] = "NONE"

# Prerequisites

Please make sure your environmental variables and dependencies are ready to use LLM services. Name of the environmental variables is arbitraray because langrila modules accepts that name as an argument.

In [2]:
from dotenv import load_dotenv

load_dotenv("../../.env_api")

True

# Import modules

Main modules are `{Provider}Client` module and `Agent` module. `{Provider}Client` module is inherited by each module for a specific llm provider like OpenAI, Google or Anthropic.

In [3]:
from langrila import Agent
from langrila.anthropic import AnthropicClient
from langrila.google import GoogleClient
from langrila.openai import OpenAIClient

# Basic usage

Instantiating client modules.

In [4]:
# For OpenAI
openai_client = OpenAIClient(api_key_env_name="OPENAI_API_KEY")

# For Azure OpenAI
azure_openai_client = OpenAIClient(
    api_key_env_name="AZURE_API_KEY",
    api_type="azure",
    azure_api_version="2024-11-01-preview",
    azure_endpoint_env_name="AZURE_ENDPOINT",
    azure_deployment_id_env_name="AZURE_DEPLOYMENT_ID",
)

# For Gemini on Google AI Studio
google_dev_client = GoogleClient(api_key_env_name="GEMINI_API_KEY")

# For Gemini on Google Cloud VertexAI
vertexai_client = GoogleClient(
    api_type="vertexai",
    project_id_env_name="GOOGLE_CLOUD_PROJECT",
    location="us-central1",
)

# For Claude of Anthropic
anthropic_client = AnthropicClient(api_key_env_name="ANTHROPIC_API_KEY")

# For Claude of Amazon Bedrock
claude_bedrock_client = AnthropicClient(
    api_type="bedrock",
    aws_access_key_env_name="AWS_ACCESS_KEY",
    aws_secret_key_env_name="AWS_SECRET_KEY",
    aws_region_env_name="AWS_REGION",
)

You can pass any client instance to `Agent`. Almost all arguments is the same as the provider's sdk. Arguments of the `Agent` class are finally passed to the `{Provider}Client` module almost as it is, and `{Provider}Client` is just simple wrapper module of each LLM provider API. So please look at the API reference of each provider API if you want to know what arguments can be accepted by `Agent`.

In [5]:
openai_agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    temperature=0.0,
)

azure_openai_agent = Agent(
    client=azure_openai_client,
    model="gpt-4o-mini-2024-07-18",
    temperature=0.0,
)

google_agent = Agent(
    client=google_dev_client,
    model="gemini-2.0-flash-exp",
    temperature=0.0,
)

vertexai_agent = Agent(
    client=vertexai_client,
    model="gemini-2.0-flash-exp",
    temperature=0.0,
)

claude_agent = Agent(
    client=anthropic_client,
    model="claude-3-5-sonnet-20240620",
    temperature=0.0,
    max_tokens=500,
)

claude_bedrock_agent = Agent(
    client=claude_bedrock_client,
    model="anthropic.claude-3-sonnet-20240229-v1:0",
    temperature=0.0,
    max_tokens=500,
)

# Generate text

## Prompt

You can input your prompt in 2 ways; input string prompt directly or create `Prompt` instance.

In [6]:
from langrila import Prompt, TextPrompt

In [7]:
# 1. Simple string prompt
prompt1 = "Hello. How are you today?"

# 2. Prompt instance with raw string
prompt2 = Prompt(
    role="user",
    contents="Hello. How are you today?",
)

# 3. Prompt instance with list of TextPrompt
prompt3 = Prompt(
    role="user",
    contents=[
        TextPrompt(
            text="Hello. How are you today?",
        ),
    ],
)

In the pattern 2, string specified as `contents` argument is automatically converted to list of `TextPrompt`.

In [8]:
prompt2 == prompt3

True

In [9]:
prompt2

Prompt(type='Prompt', role='user', contents=[TextPrompt(text='Hello. How are you today?')], name=None)

In [10]:
prompt3

Prompt(type='Prompt', role='user', contents=[TextPrompt(text='Hello. How are you today?')], name=None)

## Synchronous, non-streaming text generation

In [11]:
response = openai_agent.generate_text(prompt=prompt1)

# Same as
# response = openai_agent.generate_text(prompt=prompt2)
# response = openai_agent.generate_text(prompt=prompt3)

[2025-01-09 23:39:41] DEBUG | Prompt: [TextPrompt(text='Hello. How are you today?')]
[2025-01-09 23:39:41] INFO | root: Generating text
[2025-01-09 23:39:42] DEBUG | Response: [TextResponse(text="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?")]


response object is a pydantic model contains formatted response from the provider API, usage, and raw response.

In [12]:
response

Response(type='Response', role='assistant', contents=[TextResponse(text="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?")], usage=<langrila.core.usage.NamedUsage object at 0x7fde0435f970>, raw=ChatCompletion(id='chatcmpl-AnnzCtVoxMGxOO9Tz6sKgcQX807h5', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1736433582, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_f2cd28694a', usage=CompletionUsage(completion_tokens=30, prompt_tokens=14, total_tokens=44, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_

In [13]:
print(response.contents[0].text)

Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


Agent class accumulates usage across all API call and interaction with subagent. Sub-agent has its own name, and root agent is named "root". You can access this accumulated usage via `response.usage`. `response.usage` is dict-like object, and it has `items()` method and `__getitem__()` method.

In [14]:
response.usage.items()

dict_items([('root', Usage(model_name='gpt-4o-mini-2024-07-18', prompt_tokens=14, output_tokens=30))])

In [15]:
response.usage["root"]

Usage(model_name='gpt-4o-mini-2024-07-18', prompt_tokens=14, output_tokens=30)

You can access raw response from the API that is helpful fo9r debugging.

In [16]:
response.raw

ChatCompletion(id='chatcmpl-AnnzCtVoxMGxOO9Tz6sKgcQX807h5', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1736433582, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_f2cd28694a', usage=CompletionUsage(completion_tokens=30, prompt_tokens=14, total_tokens=44, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

Same as other client.

Azure OpenAI

In [17]:
response = azure_openai_agent.generate_text(prompt=prompt1)
response

[2025-01-09 23:39:42] DEBUG | Prompt: [TextPrompt(text='Hello. How are you today?')]
[2025-01-09 23:39:42] INFO | root: Generating text
[2025-01-09 23:39:44] DEBUG | Response: [TextResponse(text="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?")]


Response(type='Response', role='assistant', contents=[TextResponse(text="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?")], usage=<langrila.core.usage.NamedUsage object at 0x7fddd0b4afe0>, raw=ChatCompletion(id='chatcmpl-AnnzECbhlflRLvtqHbFxTJAr2qyC0', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1736433584, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_5154047bf2', usage=Co

In [18]:
response.raw

ChatCompletion(id='chatcmpl-AnnzECbhlflRLvtqHbFxTJAr2qyC0', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1736433584, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_5154047bf2', usage=CompletionUsage(completion_tokens=29, prompt_tokens=14, total_tokens=43, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(aud

Gemini on Google AI Studio

In [19]:
response = google_agent.generate_text(prompt=prompt1)
response

[2025-01-09 23:39:44] DEBUG | Prompt: [TextPrompt(text='Hello. How are you today?')]
[2025-01-09 23:39:44] INFO | root: Generating text
[2025-01-09 23:39:45] DEBUG | Response: [TextResponse(text="I'm doing well, thank you for asking! As a large language model, I don't experience feelings in the same way humans do, but I'm functioning optimally and ready to assist you. How are you doing today? Is there anything I can help you with?")]


Response(type='Response', role='assistant', contents=[TextResponse(text="I'm doing well, thank you for asking! As a large language model, I don't experience feelings in the same way humans do, but I'm functioning optimally and ready to assist you. How are you doing today? Is there anything I can help you with?")], usage=<langrila.core.usage.NamedUsage object at 0x7fddd031bf10>, raw=GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text="I'm doing well, thank you for asking! As a large language model, I don't experience feelings in the same way humans do, but I'm functioning optimally and ready to assist you. How are you doing today? Is there anything I can help you with?\n")], role='model'), citation_metadata=None, finish_message=None, token_count=None, avg_logprobs=inf, finish_reason='STOP', grounding

In [20]:
response.raw

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text="I'm doing well, thank you for asking! As a large language model, I don't experience feelings in the same way humans do, but I'm functioning optimally and ready to assist you. How are you doing today? Is there anything I can help you with?\n")], role='model'), citation_metadata=None, finish_message=None, token_count=None, avg_logprobs=inf, finish_reason='STOP', grounding_metadata=None, index=None, logprobs_result=None, safety_ratings=[SafetyRating(blocked=None, category='HARM_CATEGORY_HATE_SPEECH', probability='NEGLIGIBLE', probability_score=None, severity=None, severity_score=None), SafetyRating(blocked=None, category='HARM_CATEGORY_DANGEROUS_CONTENT', probability='NEGLIGIBLE', probability_score=None, severity=None, severity_score=None), SafetyRat

Gemini on VertexAI

In [21]:
response = vertexai_agent.generate_text(prompt=prompt1)
response

[2025-01-09 23:39:46] DEBUG | Prompt: [TextPrompt(text='Hello. How are you today?')]
[2025-01-09 23:39:46] INFO | root: Generating text
[2025-01-09 23:39:48] DEBUG | Response: [TextResponse(text="I'm doing well, thank you for asking! As a large language model, I don't experience feelings in the same way humans do, but I'm functioning optimally and ready to assist you. How are you doing today? Is there anything I can help you with?")]


Response(type='Response', role='assistant', contents=[TextResponse(text="I'm doing well, thank you for asking! As a large language model, I don't experience feelings in the same way humans do, but I'm functioning optimally and ready to assist you. How are you doing today? Is there anything I can help you with?")], usage=<langrila.core.usage.NamedUsage object at 0x7fddd0319960>, raw=GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text="I'm doing well, thank you for asking! As a large language model, I don't experience feelings in the same way humans do, but I'm functioning optimally and ready to assist you. How are you doing today? Is there anything I can help you with?\n")], role='model'), citation_metadata=None, finish_message=None, token_count=None, avg_logprobs=inf, finish_reason='STOP', grounding

In [22]:
response.raw

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text="I'm doing well, thank you for asking! As a large language model, I don't experience feelings in the same way humans do, but I'm functioning optimally and ready to assist you. How are you doing today? Is there anything I can help you with?\n")], role='model'), citation_metadata=None, finish_message=None, token_count=None, avg_logprobs=inf, finish_reason='STOP', grounding_metadata=None, index=None, logprobs_result=None, safety_ratings=None)], model_version='gemini-2.0-flash-exp', prompt_feedback=None, usage_metadata=GenerateContentResponseUsageMetadata(cached_content_token_count=None, candidates_token_count=58, prompt_token_count=7, total_token_count=65), automatic_function_calling_history=[], parsed=None)

Claude of Anthropic

In [23]:
response = claude_agent.generate_text(prompt=prompt1)
response

[2025-01-09 23:39:48] DEBUG | Prompt: [TextPrompt(text='Hello. How are you today?')]
[2025-01-09 23:39:48] INFO | root: Generating text
[2025-01-09 23:39:49] DEBUG | Response: [TextResponse(text="Hello! As an AI language model, I don't have feelings, but I'm functioning well and ready to assist you with any questions or tasks you may have. How can I help you today?")]


Response(type='Response', role='assistant', contents=[TextResponse(text="Hello! As an AI language model, I don't have feelings, but I'm functioning well and ready to assist you with any questions or tasks you may have. How can I help you today?")], usage=<langrila.core.usage.NamedUsage object at 0x7fddd0b4bb50>, raw=Message(id='msg_01VqzgRRfftfsg3i55ZMoVb7', content=[TextBlock(text="Hello! As an AI language model, I don't have feelings, but I'm functioning well and ready to assist you with any questions or tasks you may have. How can I help you today?", type='text')], model='claude-3-5-sonnet-20240620', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=14, output_tokens=43, cache_creation_input_tokens=0, cache_read_input_tokens=0)), name=None, is_last_chunk=None, prompt=[{'role': 'user', 'content': [{'text': 'Hello. How are you today?', 'type': 'text'}]}])

In [24]:
response.raw

Message(id='msg_01VqzgRRfftfsg3i55ZMoVb7', content=[TextBlock(text="Hello! As an AI language model, I don't have feelings, but I'm functioning well and ready to assist you with any questions or tasks you may have. How can I help you today?", type='text')], model='claude-3-5-sonnet-20240620', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=14, output_tokens=43, cache_creation_input_tokens=0, cache_read_input_tokens=0))

Claude on Amazon Bedrock

In [25]:
response = claude_bedrock_agent.generate_text(prompt=prompt1)
response

[2025-01-09 23:39:49] DEBUG | Prompt: [TextPrompt(text='Hello. How are you today?')]
[2025-01-09 23:39:49] INFO | root: Generating text
[2025-01-09 23:39:51] DEBUG | Response: [TextResponse(text="Hello! As an AI language model, I don't have feelings or emotions, but I'm operating properly and ready to assist you with any questions or tasks you may have. How can I help you today?")]


Response(type='Response', role='assistant', contents=[TextResponse(text="Hello! As an AI language model, I don't have feelings or emotions, but I'm operating properly and ready to assist you with any questions or tasks you may have. How can I help you today?")], usage=<langrila.core.usage.NamedUsage object at 0x7fddd031b9a0>, raw=Message(id='msg_bdrk_01KnPF2BTRkRPzcQwHMMBRx8', content=[TextBlock(text="Hello! As an AI language model, I don't have feelings or emotions, but I'm operating properly and ready to assist you with any questions or tasks you may have. How can I help you today?", type='text')], model='claude-3-sonnet-20240229', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=14, output_tokens=45)), name=None, is_last_chunk=None, prompt=[{'role': 'user', 'content': [{'text': 'Hello. How are you today?', 'type': 'text'}]}])

In [26]:
response.raw

Message(id='msg_bdrk_01KnPF2BTRkRPzcQwHMMBRx8', content=[TextBlock(text="Hello! As an AI language model, I don't have feelings or emotions, but I'm operating properly and ready to assist you with any questions or tasks you may have. How can I help you today?", type='text')], model='claude-3-sonnet-20240229', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=14, output_tokens=45))

## Constructor params vs generation params

Except a few parameters, almost all the parameters can be specified in both constructor and generation method (such as `generate_text()` method). Constructor params in `Agent` class can be overriden by the arguments in `generate_text` method (as well as `stream_text(_async)` method as shown later).

In [27]:
agent = Agent(
    client=google_dev_client,
    model="gemini-2.0-flash-exp",
    temperature=0.0,
    max_output_tokens=500,
)

agent.generate_text(prompt="Hello!")

[2025-01-09 23:39:51] DEBUG | Prompt: [TextPrompt(text='Hello!')]
[2025-01-09 23:39:51] INFO | root: Generating text
[2025-01-09 23:39:52] DEBUG | Response: [TextResponse(text='Hello there! How can I help you today?')]


Response(type='Response', role='assistant', contents=[TextResponse(text='Hello there! How can I help you today?')], usage=<langrila.core.usage.NamedUsage object at 0x7fddcff5cd00>, raw=GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='Hello there! How can I help you today?\n')], role='model'), citation_metadata=None, finish_message=None, token_count=None, avg_logprobs=inf, finish_reason='STOP', grounding_metadata=None, index=None, logprobs_result=None, safety_ratings=[SafetyRating(blocked=None, category='HARM_CATEGORY_HATE_SPEECH', probability='NEGLIGIBLE', probability_score=None, severity=None, severity_score=None), SafetyRating(blocked=None, category='HARM_CATEGORY_DANGEROUS_CONTENT', probability='NEGLIGIBLE', probability_score=None, severity=None, severity_score=None), SafetyRating(blocked=Non

is the same as:

In [28]:
agent = Agent(
    client=google_dev_client,
)

agent.generate_text(
    prompt="Hello!", model="gemini-2.0-flash-exp", temperature=0.0, max_output_tokens=500
)

[2025-01-09 23:39:52] DEBUG | Prompt: [TextPrompt(text='Hello!')]
[2025-01-09 23:39:52] INFO | root: Generating text
[2025-01-09 23:39:54] DEBUG | Response: [TextResponse(text='Hello there! How can I help you today?')]


Response(type='Response', role='assistant', contents=[TextResponse(text='Hello there! How can I help you today?')], usage=<langrila.core.usage.NamedUsage object at 0x7fddd0b4b820>, raw=GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='Hello there! How can I help you today?\n')], role='model'), citation_metadata=None, finish_message=None, token_count=None, avg_logprobs=inf, finish_reason='STOP', grounding_metadata=None, index=None, logprobs_result=None, safety_ratings=[SafetyRating(blocked=None, category='HARM_CATEGORY_HATE_SPEECH', probability='NEGLIGIBLE', probability_score=None, severity=None, severity_score=None), SafetyRating(blocked=None, category='HARM_CATEGORY_DANGEROUS_CONTENT', probability='NEGLIGIBLE', probability_score=None, severity=None, severity_score=None), SafetyRating(blocked=Non

Passing arguments when initiating is useful to reuse same parameters at multiple place. Especiall, multi-agent system invokes multiple agents at the same time, so it requires many parameters to be static. On the other hand, passing arguments when generating helps you control settings whenever you call the API. This is helpful for the single agent case.

Then, here is an example to specify the same parameter when initializing agent and generating response. (As I mentioned, the later parameter is prioritized.)

In [29]:
agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    n=1,
)

In [30]:
response = agent.generate_text(prompt="Hello! How are you?", n=2)

[2025-01-09 23:39:54] DEBUG | Prompt: [TextPrompt(text='Hello! How are you?')]
[2025-01-09 23:39:54] INFO | root: Generating text
[2025-01-09 23:39:55] DEBUG | Response: [TextResponse(text="Hello! I'm just a computer program, but I'm here and ready to help you. How can I assist you today?"), TextResponse(text="Hello! I'm just a computer program, but I'm here and ready to help you. How can I assist you today?")]


In [31]:
response.contents

[TextResponse(text="Hello! I'm just a computer program, but I'm here and ready to help you. How can I assist you today?"),
 TextResponse(text="Hello! I'm just a computer program, but I'm here and ready to help you. How can I assist you today?")]

In this case, `n` parameter is specified in both constructor and `generate_text` method, but `n=2` is used, which was specified for `generate_text` method.

## Asynchronous, non-streaming text generation

`generate_text_async` method allows you to received text asyncronously.

OpenAI

In [32]:
response = await openai_agent.generate_text_async(prompt=prompt1)
response

[2025-01-09 23:39:55] DEBUG | Prompt: [TextPrompt(text='Hello. How are you today?')]
[2025-01-09 23:39:55] INFO | root: Generating text
[2025-01-09 23:39:56] DEBUG | Response: [TextResponse(text="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?")]


Response(type='Response', role='assistant', contents=[TextResponse(text="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?")], usage=<langrila.core.usage.NamedUsage object at 0x7fddcff4fdc0>, raw=ChatCompletion(id='chatcmpl-AnnzQXiQPjI0BfNhkl9ITiU45yR6l', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1736433596, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_f2cd28694a', usage=CompletionUsage(completion_tokens=30, prompt_tokens=14, total_tokens=44, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_

Azure OpenAI

In [33]:
response = await azure_openai_agent.generate_text_async(prompt=prompt1)
response

[2025-01-09 23:39:56] DEBUG | Prompt: [TextPrompt(text='Hello. How are you today?')]
[2025-01-09 23:39:56] INFO | root: Generating text
[2025-01-09 23:39:57] DEBUG | Response: [TextResponse(text="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?")]


Response(type='Response', role='assistant', contents=[TextResponse(text="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?")], usage=<langrila.core.usage.NamedUsage object at 0x7fddcff76c50>, raw=ChatCompletion(id='chatcmpl-AnnzRpgyw3GV7WoujspcULhoRGEnv', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1736433597, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_f3927aa00d', usage=Co

Gemini

In [34]:
response = await google_agent.generate_text_async(prompt=prompt1)
response

[2025-01-09 23:39:57] DEBUG | Prompt: [TextPrompt(text='Hello. How are you today?')]
[2025-01-09 23:39:57] INFO | root: Generating text
[2025-01-09 23:39:59] DEBUG | Response: [TextResponse(text="I'm doing well, thank you for asking! As a large language model, I don't experience feelings in the same way humans do, but I'm functioning optimally and ready to assist you. How are you doing today? Is there anything I can help you with?")]


Response(type='Response', role='assistant', contents=[TextResponse(text="I'm doing well, thank you for asking! As a large language model, I don't experience feelings in the same way humans do, but I'm functioning optimally and ready to assist you. How are you doing today? Is there anything I can help you with?")], usage=<langrila.core.usage.NamedUsage object at 0x7fddd03f17b0>, raw=GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text="I'm doing well, thank you for asking! As a large language model, I don't experience feelings in the same way humans do, but I'm functioning optimally and ready to assist you. How are you doing today? Is there anything I can help you with?\n")], role='model'), citation_metadata=None, finish_message=None, token_count=None, avg_logprobs=inf, finish_reason='STOP', grounding

Gemini on VertexAI

In [35]:
response = await vertexai_agent.generate_text_async(prompt=prompt1)
response

[2025-01-09 23:39:59] DEBUG | Prompt: [TextPrompt(text='Hello. How are you today?')]
[2025-01-09 23:39:59] INFO | root: Generating text
[2025-01-09 23:40:01] DEBUG | Response: [TextResponse(text="I'm doing well, thank you for asking! As a large language model, I don't experience feelings in the same way humans do, but I'm functioning optimally and ready to assist you. How are you doing today? Is there anything I can help you with?")]


Response(type='Response', role='assistant', contents=[TextResponse(text="I'm doing well, thank you for asking! As a large language model, I don't experience feelings in the same way humans do, but I'm functioning optimally and ready to assist you. How are you doing today? Is there anything I can help you with?")], usage=<langrila.core.usage.NamedUsage object at 0x7fddcff4f820>, raw=GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text="I'm doing well, thank you for asking! As a large language model, I don't experience feelings in the same way humans do, but I'm functioning optimally and ready to assist you. How are you doing today? Is there anything I can help you with?\n")], role='model'), citation_metadata=None, finish_message=None, token_count=None, avg_logprobs=inf, finish_reason='STOP', grounding

Claude of Anthropic

In [36]:
response = await claude_agent.generate_text_async(prompt=prompt1)
response

[2025-01-09 23:40:01] DEBUG | Prompt: [TextPrompt(text='Hello. How are you today?')]
[2025-01-09 23:40:01] INFO | root: Generating text
[2025-01-09 23:40:02] DEBUG | Response: [TextResponse(text="Hello! As an AI language model, I don't have feelings, but I'm functioning well and ready to assist you with any questions or tasks you may have. How can I help you today?")]


Response(type='Response', role='assistant', contents=[TextResponse(text="Hello! As an AI language model, I don't have feelings, but I'm functioning well and ready to assist you with any questions or tasks you may have. How can I help you today?")], usage=<langrila.core.usage.NamedUsage object at 0x7fddcff5e470>, raw=Message(id='msg_01RcJHMaaggSrE5Xm9htamEE', content=[TextBlock(text="Hello! As an AI language model, I don't have feelings, but I'm functioning well and ready to assist you with any questions or tasks you may have. How can I help you today?", type='text')], model='claude-3-5-sonnet-20240620', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=14, output_tokens=43, cache_creation_input_tokens=0, cache_read_input_tokens=0)), name=None, is_last_chunk=None, prompt=[{'role': 'user', 'content': [{'text': 'Hello. How are you today?', 'type': 'text'}]}])

Claude of Amazon Bedrock

In [37]:
response = await claude_bedrock_agent.generate_text_async(prompt=prompt1)
response

[2025-01-09 23:40:02] DEBUG | Prompt: [TextPrompt(text='Hello. How are you today?')]
[2025-01-09 23:40:02] INFO | root: Generating text
[2025-01-09 23:40:03] DEBUG | Response: [TextResponse(text="Hello! As an AI language model, I don't have feelings or emotions, but I'm operating properly and ready to assist you with any questions or tasks you may have. How can I help you today?")]


Response(type='Response', role='assistant', contents=[TextResponse(text="Hello! As an AI language model, I don't have feelings or emotions, but I'm operating properly and ready to assist you with any questions or tasks you may have. How can I help you today?")], usage=<langrila.core.usage.NamedUsage object at 0x7fddcffdb550>, raw=Message(id='msg_bdrk_01DgyEk9dEEaFfG8v3HYuy9v', content=[TextBlock(text="Hello! As an AI language model, I don't have feelings or emotions, but I'm operating properly and ready to assist you with any questions or tasks you may have. How can I help you today?", type='text')], model='claude-3-sonnet-20240229', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=14, output_tokens=45)), name=None, is_last_chunk=None, prompt=[{'role': 'user', 'content': [{'text': 'Hello. How are you today?', 'type': 'text'}]}])

## Synchronous streaming text generation

OpenAI

In [38]:
streamed_response = openai_agent.stream_text(prompt=prompt1)

for chunk in streamed_response:
    print(chunk.contents[0].text, flush=True)

[2025-01-09 23:40:04] DEBUG | Prompt: [TextPrompt(text='Hello. How are you today?')]
[2025-01-09 23:40:04] INFO | root: Generating text


Hello
Hello!
Hello! I'm
Hello! I'm just
Hello! I'm just a
Hello! I'm just a program
Hello! I'm just a program,
Hello! I'm just a program, so
Hello! I'm just a program, so I
Hello! I'm just a program, so I don't
Hello! I'm just a program, so I don't have
Hello! I'm just a program, so I don't have feelings
Hello! I'm just a program, so I don't have feelings,
Hello! I'm just a program, so I don't have feelings, but
Hello! I'm just a program, so I don't have feelings, but I'm
Hello! I'm just a program, so I don't have feelings, but I'm here
Hello! I'm just a program, so I don't have feelings, but I'm here and
Hello! I'm just a program, so I don't have feelings, but I'm here and ready
Hello! I'm just a program, so I don't have feelings, but I'm here and ready to
Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help
Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you
Hello! I'm just a program, so I don't have feelings, but I'm

[2025-01-09 23:40:04] DEBUG | Response: [TextResponse(text="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?")]


Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


Each chunk are the combination of the current and past chunks, and `stream_text` method is designed to return the result finally same as the response from the `generate_text` method.

In [39]:
chunk

Response(type='Response', role='assistant', contents=[TextResponse(text="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?")], usage=<langrila.core.usage.NamedUsage object at 0x7fddcffda980>, raw=<openai.Stream object at 0x7fddcff5dea0>, name=None, is_last_chunk=True, prompt=None)

In [40]:
chunk.usage.items()

dict_items([('root', Usage(model_name='gpt-4o-mini-2024-07-18', prompt_tokens=14, output_tokens=29))])

Other client has the same interface.

Azure OpenAI

In [41]:
streamed_response = azure_openai_agent.stream_text(prompt=prompt1)

for chunk in streamed_response:
    print(chunk.contents[0].text, flush=True)

[2025-01-09 23:40:04] DEBUG | Prompt: [TextPrompt(text='Hello. How are you today?')]
[2025-01-09 23:40:04] INFO | root: Generating text


Hello
Hello!
Hello! I'm
Hello! I'm just
Hello! I'm just a
Hello! I'm just a program
Hello! I'm just a program,
Hello! I'm just a program, so
Hello! I'm just a program, so I
Hello! I'm just a program, so I don't
Hello! I'm just a program, so I don't have
Hello! I'm just a program, so I don't have feelings
Hello! I'm just a program, so I don't have feelings,
Hello! I'm just a program, so I don't have feelings, but
Hello! I'm just a program, so I don't have feelings, but I'm
Hello! I'm just a program, so I don't have feelings, but I'm here
Hello! I'm just a program, so I don't have feelings, but I'm here and
Hello! I'm just a program, so I don't have feelings, but I'm here and ready
Hello! I'm just a program, so I don't have feelings, but I'm here and ready to
Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help
Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you
Hello! I'm just a program, so I don't have feelings, but I'm

[2025-01-09 23:40:08] DEBUG | Response: [TextResponse(text="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?")]


Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


In [42]:
chunk

Response(type='Response', role='assistant', contents=[TextResponse(text="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?")], usage=<langrila.core.usage.NamedUsage object at 0x7fddcff5e3b0>, raw=<openai.Stream object at 0x7fddd031b940>, name=None, is_last_chunk=True, prompt=None)

Gemini on Google AI Studio

In [43]:
streamed_response = google_agent.stream_text(prompt=prompt1)

for chunk in streamed_response:
    print(chunk.contents[0].text, flush=True)

[2025-01-09 23:40:08] DEBUG | Prompt: [TextPrompt(text='Hello. How are you today?')]
[2025-01-09 23:40:08] INFO | root: Generating text


I
I'm doing well, thank you for asking! As a large language model,
I'm doing well, thank you for asking! As a large language model, I don't experience feelings in the same way humans do, but I'
I'm doing well, thank you for asking! As a large language model, I don't experience feelings in the same way humans do, but I'm functioning optimally and ready to assist you. How are you doing today? Is there anything I can help you with?

I'm doing well, thank you for asking! As a large language model, I don't experience feelings in the same way humans do, but I'm functioning optimally and ready to assist you. How are you doing today? Is there anything I can help you with?



In [44]:
chunk

Response(type='Response', role='assistant', contents=[TextResponse(text="I'm doing well, thank you for asking! As a large language model, I don't experience feelings in the same way humans do, but I'm functioning optimally and ready to assist you. How are you doing today? Is there anything I can help you with?\n")], usage=<langrila.core.usage.NamedUsage object at 0x7fddcffefc70>, raw=GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='m functioning optimally and ready to assist you. How are you doing today? Is there anything I can help you with?\n')], role='model'), citation_metadata=None, finish_message=None, token_count=None, avg_logprobs=None, finish_reason='STOP', grounding_metadata=None, index=None, logprobs_result=None, safety_ratings=[SafetyRating(blocked=None, category='HARM_CATEGORY_HATE_S

Gemini on VertexAI

In [45]:
streamed_response = vertexai_agent.stream_text(prompt=prompt1)

for chunk in streamed_response:
    print(chunk.contents[0].text, flush=True)

[2025-01-09 23:40:09] DEBUG | Prompt: [TextPrompt(text='Hello. How are you today?')]
[2025-01-09 23:40:09] INFO | root: Generating text


I
I'm doing well, thank you for asking! As a large language model,
I'm doing well, thank you for asking! As a large language model, I don't experience feelings in the same way humans do, but I'
I'm doing well, thank you for asking! As a large language model, I don't experience feelings in the same way humans do, but I'm functioning optimally and ready to assist you. How are you doing today? Is there anything I can help you with?

I'm doing well, thank you for asking! As a large language model, I don't experience feelings in the same way humans do, but I'm functioning optimally and ready to assist you. How are you doing today? Is there anything I can help you with?



In [46]:
chunk

Response(type='Response', role='assistant', contents=[TextResponse(text="I'm doing well, thank you for asking! As a large language model, I don't experience feelings in the same way humans do, but I'm functioning optimally and ready to assist you. How are you doing today? Is there anything I can help you with?\n")], usage=<langrila.core.usage.NamedUsage object at 0x7fddcffefeb0>, raw=GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='m functioning optimally and ready to assist you. How are you doing today? Is there anything I can help you with?\n')], role='model'), citation_metadata=None, finish_message=None, token_count=None, avg_logprobs=None, finish_reason='STOP', grounding_metadata=None, index=None, logprobs_result=None, safety_ratings=None)], model_version='gemini-2.0-flash-exp', prompt_feedb

Claude on Anthropic

In [47]:
streamed_response = claude_agent.stream_text(prompt=prompt1)

for chunk in streamed_response:
    print(chunk.contents[0].text, flush=True)

[2025-01-09 23:40:11] DEBUG | Prompt: [TextPrompt(text='Hello. How are you today?')]
[2025-01-09 23:40:11] INFO | root: Generating text


Hello! As
Hello! As an AI language model, I don't have feelings
Hello! As an AI language model, I don't have feelings, but I'm functioning well and ready to assist you with any questions or
Hello! As an AI language model, I don't have feelings, but I'm functioning well and ready to assist you with any questions or tasks you may have. How can I help you today?


[2025-01-09 23:40:13] DEBUG | Response: [TextResponse(text="Hello! As an AI language model, I don't have feelings, but I'm functioning well and ready to assist you with any questions or tasks you may have. How can I help you today?")]


Hello! As an AI language model, I don't have feelings, but I'm functioning well and ready to assist you with any questions or tasks you may have. How can I help you today?


In [48]:
chunk

Response(type='Response', role='assistant', contents=[TextResponse(text="Hello! As an AI language model, I don't have feelings, but I'm functioning well and ready to assist you with any questions or tasks you may have. How can I help you today?")], usage=<langrila.core.usage.NamedUsage object at 0x7fddcffef7c0>, raw=None, name=None, is_last_chunk=True, prompt=None)

Claude on Bedrock

In [49]:
streamed_response = claude_bedrock_agent.stream_text(prompt=prompt1)

for chunk in streamed_response:
    print(chunk.contents[0].text, flush=True)

[2025-01-09 23:40:13] DEBUG | Prompt: [TextPrompt(text='Hello. How are you today?')]
[2025-01-09 23:40:13] INFO | root: Generating text


Hello
Hello!
Hello! As
Hello! As an
Hello! As an AI
Hello! As an AI language
Hello! As an AI language model
Hello! As an AI language model,
Hello! As an AI language model, I
Hello! As an AI language model, I don
Hello! As an AI language model, I don't
Hello! As an AI language model, I don't have
Hello! As an AI language model, I don't have feelings
Hello! As an AI language model, I don't have feelings or
Hello! As an AI language model, I don't have feelings or emotions
Hello! As an AI language model, I don't have feelings or emotions,
Hello! As an AI language model, I don't have feelings or emotions, but
Hello! As an AI language model, I don't have feelings or emotions, but I
Hello! As an AI language model, I don't have feelings or emotions, but I'm
Hello! As an AI language model, I don't have feelings or emotions, but I'm operating
Hello! As an AI language model, I don't have feelings or emotions, but I'm operating properly
Hello! As an AI language model, I don't have feelings or emot

[2025-01-09 23:40:16] DEBUG | Response: [TextResponse(text="Hello! As an AI language model, I don't have feelings or emotions, but I'm operating properly and ready to assist you with any questions or tasks you may have. How can I help you today?")]


Hello! As an AI language model, I don't have feelings or emotions, but I'm operating properly and ready to assist you with any questions or tasks you may have. How can I help you today?


In [50]:
chunk

Response(type='Response', role='assistant', contents=[TextResponse(text="Hello! As an AI language model, I don't have feelings or emotions, but I'm operating properly and ready to assist you with any questions or tasks you may have. How can I help you today?")], usage=<langrila.core.usage.NamedUsage object at 0x7fddcff4f7c0>, raw=None, name=None, is_last_chunk=True, prompt=None)

## Asynchronously streaming text generation

`stream_text_async` method is responsible for asynchronous streaming text generation.

OpenAI

In [51]:
streamed_response = openai_agent.stream_text_async(prompt=prompt1)

async for chunk in streamed_response:
    print(chunk.contents[0].text, flush=True)

[2025-01-09 23:40:16] DEBUG | Prompt: [TextPrompt(text='Hello. How are you today?')]
[2025-01-09 23:40:16] INFO | root: Generating text


Hello
Hello!
Hello! I'm
Hello! I'm just
Hello! I'm just a
Hello! I'm just a program
Hello! I'm just a program,
Hello! I'm just a program, so
Hello! I'm just a program, so I
Hello! I'm just a program, so I don't
Hello! I'm just a program, so I don't have
Hello! I'm just a program, so I don't have feelings
Hello! I'm just a program, so I don't have feelings,
Hello! I'm just a program, so I don't have feelings, but
Hello! I'm just a program, so I don't have feelings, but I'm
Hello! I'm just a program, so I don't have feelings, but I'm here
Hello! I'm just a program, so I don't have feelings, but I'm here and
Hello! I'm just a program, so I don't have feelings, but I'm here and ready
Hello! I'm just a program, so I don't have feelings, but I'm here and ready to
Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help
Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you
Hello! I'm just a program, so I don't have feelings, but I'm

[2025-01-09 23:40:17] DEBUG | Response: [TextResponse(text="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?")]


Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


In [52]:
chunk

Response(type='Response', role='assistant', contents=[TextResponse(text="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?")], usage=<langrila.core.usage.NamedUsage object at 0x7fddcffef940>, raw=<openai.AsyncStream object at 0x7fddcff76860>, name=None, is_last_chunk=True, prompt=None)

Same as other client, so skip demonstration.

# System prompt

In langrila, every client accepts `system_instruction` argument. This argument can be specified in both constructor and `generate_text(_async)`/`stream_text(_async)` method. The later case overrides constructor's system_instruction. System instruction needs to be `SystemPrompt` instance because OpenAI API needs role property.

In [53]:
from langrila import SystemPrompt

system_prompt = SystemPrompt(
    role="system",  # or developer role used for the models later than o1 for OpenAI API.
    contents="You must to answer the question in Japanese.",
)

OpenAI

In [54]:
response = openai_agent.generate_text(prompt=prompt1, system_instruction=system_prompt)
response

[2025-01-09 23:40:17] DEBUG | Prompt: [TextPrompt(text='Hello. How are you today?')]
[2025-01-09 23:40:17] INFO | root: Generating text
[2025-01-09 23:40:18] DEBUG | Response: [TextResponse(text='こんにちは。今日は元気です。あなたはいかがですか？')]


Response(type='Response', role='assistant', contents=[TextResponse(text='こんにちは。今日は元気です。あなたはいかがですか？')], usage=<langrila.core.usage.NamedUsage object at 0x7fddcffda230>, raw=ChatCompletion(id='chatcmpl-AnnzmmaMQQ9jiaaQEnMmqOL9TbuYU', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='こんにちは。今日は元気です。あなたはいかがですか？', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1736433618, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_0aa8d3e20b', usage=CompletionUsage(completion_tokens=15, prompt_tokens=27, total_tokens=42, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))), name=None, is_last_chunk=None, prompt=[{'role': 'system', 'content': 'You must to answer the question in Japanese.'}, 

Azure OpenAI

In [55]:
response = azure_openai_agent.generate_text(prompt=prompt1, system_instruction=system_prompt)
response

[2025-01-09 23:40:18] DEBUG | Prompt: [TextPrompt(text='Hello. How are you today?')]
[2025-01-09 23:40:18] INFO | root: Generating text
[2025-01-09 23:40:19] DEBUG | Response: [TextResponse(text='こんにちは。今日は元気です。あなたはいかがですか？')]


Response(type='Response', role='assistant', contents=[TextResponse(text='こんにちは。今日は元気です。あなたはいかがですか？')], usage=<langrila.core.usage.NamedUsage object at 0x7fddcffdb2b0>, raw=ChatCompletion(id='chatcmpl-Annzo2wJSRTzar4G1wlOyE84qboJL', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='こんにちは。今日は元気です。あなたはいかがですか？', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1736433620, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_5154047bf2', usage=CompletionUsage(completion_tokens=14, prompt_tokens=27, total_tokens=41, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_token

Gemini on Google AI Studio

In [56]:
response = google_agent.generate_text(prompt=prompt1, system_instruction=system_prompt)
response

[2025-01-09 23:40:19] DEBUG | Prompt: [TextPrompt(text='Hello. How are you today?')]
[2025-01-09 23:40:19] INFO | root: Generating text
[2025-01-09 23:40:21] DEBUG | Response: [TextResponse(text='こんにちは。今日は元気ですか？')]


Response(type='Response', role='assistant', contents=[TextResponse(text='こんにちは。今日は元気ですか？')], usage=<langrila.core.usage.NamedUsage object at 0x7fddcff4fe20>, raw=GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='こんにちは。今日は元気ですか？\n')], role='model'), citation_metadata=None, finish_message=None, token_count=None, avg_logprobs=inf, finish_reason='STOP', grounding_metadata=None, index=None, logprobs_result=None, safety_ratings=[SafetyRating(blocked=None, category='HARM_CATEGORY_HATE_SPEECH', probability='NEGLIGIBLE', probability_score=None, severity=None, severity_score=None), SafetyRating(blocked=None, category='HARM_CATEGORY_DANGEROUS_CONTENT', probability='NEGLIGIBLE', probability_score=None, severity=None, severity_score=None), SafetyRating(blocked=None, category='HARM_CATEGORY_HARASSMENT', probab

Gemini on VertexAI

In [57]:
response = vertexai_agent.generate_text(prompt=prompt1, system_instruction=system_prompt)
response

[2025-01-09 23:40:21] DEBUG | Prompt: [TextPrompt(text='Hello. How are you today?')]
[2025-01-09 23:40:21] INFO | root: Generating text
[2025-01-09 23:40:22] DEBUG | Response: [TextResponse(text='こんにちは。今日は元気ですか？')]


Response(type='Response', role='assistant', contents=[TextResponse(text='こんにちは。今日は元気ですか？')], usage=<langrila.core.usage.NamedUsage object at 0x7fddd031bbb0>, raw=GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='こんにちは。今日は元気ですか？\n')], role='model'), citation_metadata=None, finish_message=None, token_count=None, avg_logprobs=inf, finish_reason='STOP', grounding_metadata=None, index=None, logprobs_result=None, safety_ratings=None)], model_version='gemini-2.0-flash-exp', prompt_feedback=None, usage_metadata=GenerateContentResponseUsageMetadata(cached_content_token_count=None, candidates_token_count=7, prompt_token_count=16, total_token_count=23), automatic_function_calling_history=[], parsed=None), name=None, is_last_chunk=None, prompt=None)

Claude on Anthropic

In [58]:
response = claude_agent.generate_text(prompt=prompt1, system_instruction=system_prompt)
response

[2025-01-09 23:40:22] DEBUG | Prompt: [TextPrompt(text='Hello. How are you today?')]
[2025-01-09 23:40:22] INFO | root: Generating text
[2025-01-09 23:40:23] DEBUG | Response: [TextResponse(text='こんにちは。お元気ですか？私は人工知能のアシスタントですが、日本語でお答えするようにとの指示を受けましたので、日本語でお返事いたします。今日のお天気はいかがですか？')]


Response(type='Response', role='assistant', contents=[TextResponse(text='こんにちは。お元気ですか？私は人工知能のアシスタントですが、日本語でお答えするようにとの指示を受けましたので、日本語でお返事いたします。今日のお天気はいかがですか？')], usage=<langrila.core.usage.NamedUsage object at 0x7fddcffef7f0>, raw=Message(id='msg_01CmgNUUotkRqz7yCESWRVfy', content=[TextBlock(text='こんにちは。お元気ですか？私は人工知能のアシスタントですが、日本語でお答えするようにとの指示を受けましたので、日本語でお返事いたします。今日のお天気はいかがですか？', type='text')], model='claude-3-5-sonnet-20240620', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=23, output_tokens=76, cache_creation_input_tokens=0, cache_read_input_tokens=0)), name=None, is_last_chunk=None, prompt=[{'role': 'user', 'content': [{'text': 'Hello. How are you today?', 'type': 'text'}]}])

Claude on Bedrock

In [59]:
response = claude_bedrock_agent.generate_text(prompt=prompt1, system_instruction=system_prompt)
response

[2025-01-09 23:40:23] DEBUG | Prompt: [TextPrompt(text='Hello. How are you today?')]
[2025-01-09 23:40:23] INFO | root: Generating text
[2025-01-09 23:40:27] DEBUG | Response: [TextResponse(text='こんにちは。私は人工知能のAssistantですので、気分というものはありませんが、きちんと動作しています。何かお手伝いできることがあれば、喜んでお答えします。')]


Response(type='Response', role='assistant', contents=[TextResponse(text='こんにちは。私は人工知能のAssistantですので、気分というものはありませんが、きちんと動作しています。何かお手伝いできることがあれば、喜んでお答えします。')], usage=<langrila.core.usage.NamedUsage object at 0x7fddcf6b69b0>, raw=Message(id='msg_bdrk_01WcgsM5ewBfGyRgYBFHFZsH', content=[TextBlock(text='こんにちは。私は人工知能のAssistantですので、気分というものはありませんが、きちんと動作しています。何かお手伝いできることがあれば、喜んでお答えします。', type='text')], model='claude-3-sonnet-20240229', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=23, output_tokens=64)), name=None, is_last_chunk=None, prompt=[{'role': 'user', 'content': [{'text': 'Hello. How are you today?', 'type': 'text'}]}])

# Response schema using tool calling

Response schema in langrila is just tool calling. Generated args are validated with pydantic, then is finally output as text response. This feature is supported by the `response_schema_as_tool` argument.

In [60]:
from enum import Enum

from pydantic import BaseModel, Field


class Language(str, Enum):
    japanese = "Japanese"
    english = "English"
    french = "French"
    german = "German"


class GreetingTime(str, Enum):
    morning = "morning"
    afternoon = "afternoon"
    evening = "evening"


class Greeting(BaseModel):
    language: Language = Field(..., title="Language", description="Language to greet in.")
    greeting: str = Field(..., title="Greeting", description="Greeting message.")
    time: GreetingTime = Field(..., title="Time", description="Time of the day.")

In [61]:
prompt = """Tell me how to greet in a language. Please pick one language from Japanese, English, French, and German, then
tell me the greeting message and the time of the day to use it.
"""

In [62]:
agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    response_schema_as_tool=Greeting,  # <- This is the arg to use the schema as a tool.
)

response = agent.generate_text(prompt=prompt)

[2025-01-09 23:40:27] DEBUG | Prompt: [TextPrompt(text='Tell me how to greet in a language. Please pick one language from Japanese, English, French, and German, then\ntell me the greeting message and the time of the day to use it.\n')]
[2025-01-09 23:40:27] INFO | root: Generating text
[2025-01-09 23:40:29] DEBUG | Response: [ToolCallResponse(name='final_answer', args='{"language": "Japanese", "greeting": "おはようございます", "time": "morning"}', call_id='call_6qWUSr2itA9NahF3czfW8sFq'), ToolCallResponse(name='final_answer', args='{"language": "English", "greeting": "Good Afternoon", "time": "afternoon"}', call_id='call_CNIBl0SbKJwGAI4lyoBUynvE'), ToolCallResponse(name='final_answer', args='{"language": "French", "greeting": "Bonsoir", "time": "evening"}', call_id='call_SHleO1EkLc9u11qBoFJbZSkF'), ToolCallResponse(name='final_answer', args='{"language": "German", "greeting": "Guten Morgen", "time": "morning"}', call_id='call_FolgQPkj5TsZJbLCsTCUIPti')]
[2025-01-09 23:40:29] INFO | Running tool

In [63]:
Greeting.model_validate_json(response.contents[0].text)

Greeting(language=<Language.german: 'German'>, greeting='Guten Morgen', time=<GreetingTime.morning: 'morning'>)

`response_schema_as_tool` argument can be specified in generation method as well. If you pass this argument in both constructor and generation method, the later is used.

In [64]:
prompt = """Tell me how to greet in a language. Please pick one language from Japanese, English, French, and German, then
tell me the greeting message and the time of the day to use it.
"""

agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
)

response = agent.generate_text(prompt=prompt, response_schema_as_tool=Greeting)

[2025-01-09 23:40:29] DEBUG | Prompt: [TextPrompt(text='Tell me how to greet in a language. Please pick one language from Japanese, English, French, and German, then\ntell me the greeting message and the time of the day to use it.\n')]
[2025-01-09 23:40:29] INFO | root: Generating text
[2025-01-09 23:40:30] DEBUG | Response: [TextResponse(text='Let\'s choose Japanese. A common greeting in Japanese is "おはようございます" (Ohayō gozaimasu), which means "Good morning." You would use this greeting in the morning.')]
[2025-01-09 23:40:30] DEBUG | Prompt: [TextPrompt(text="Decide the next action based on the conversation. If you have all information for answering the user, run 'final_result' tool. If you need more information, invoke other tool to get necessary information. ")]
[2025-01-09 23:40:30] INFO | root: Generating text
[2025-01-09 23:40:32] DEBUG | Response: [ToolCallResponse(name='final_answer', args='{"language":"Japanese","greeting":"おはようございます","time":"morning"}', call_id='call_HWgUhDLAF

In [65]:
Greeting.model_validate_json(response.contents[0].text)

Greeting(language=<Language.japanese: 'Japanese'>, greeting='おはようございます', time=<GreetingTime.morning: 'morning'>)

Unless you specify the response schema when instantiating the agent, response schema is reset at every execution.

In [66]:
response = agent.generate_text(prompt=prompt)

print(response.contents[0].text)

[2025-01-09 23:40:32] DEBUG | Prompt: [TextPrompt(text='Tell me how to greet in a language. Please pick one language from Japanese, English, French, and German, then\ntell me the greeting message and the time of the day to use it.\n')]
[2025-01-09 23:40:32] INFO | root: Generating text
[2025-01-09 23:40:33] DEBUG | Response: [TextResponse(text='Let\'s choose **French**. \n\nA common greeting is **"Bonjour."** \n\nYou would use "Bonjour" during the daytime, typically until the early evening (around 5 PM). It means "Good morning" or "Good day." \n\nIn the evening, you can switch to **"Bonsoir,"** which means "Good evening."')]


Let's choose **French**. 

A common greeting is **"Bonjour."** 

You would use "Bonjour" during the daytime, typically until the early evening (around 5 PM). It means "Good morning" or "Good day." 

In the evening, you can switch to **"Bonsoir,"** which means "Good evening."


Response schema as a tool is named `final_answer`, and its default description is:

In [67]:
from langrila import AgentConfig

agent_config = AgentConfig()
print(agent_config.final_answer_description)

The final answer which ends this conversation. Arguments of this tool must be selected from the conversation history.
Unkown argument in the entire conversation history must be null, however, the argument appeared in the previous conversation must be provided.



This parameter is configurable, so if you want to use another description, please change the description.

In [68]:
agent_config = AgentConfig(
    final_answer_description="This is example of final answer description.",
)

_agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    response_schema_as_tool=Greeting,
    agent_config=agent_config,  # Set the agent configuration
)

For Gemini

In [69]:
agent = Agent(
    client=google_dev_client,
    model="gemini-2.0-flash-exp",
    response_schema_as_tool=Greeting,
)

response = agent.generate_text(prompt=prompt)

[2025-01-09 23:40:34] DEBUG | Prompt: [TextPrompt(text='Tell me how to greet in a language. Please pick one language from Japanese, English, French, and German, then\ntell me the greeting message and the time of the day to use it.\n')]
[2025-01-09 23:40:34] INFO | root: Generating text
[2025-01-09 23:40:35] DEBUG | Response: [TextResponse(text="I can help with that! Please specify which language you'd like to use.")]
[2025-01-09 23:40:35] DEBUG | Prompt: [TextPrompt(text="Decide the next action based on the conversation. If you have all information for answering the user, run 'final_result' tool. If you need more information, invoke other tool to get necessary information. ")]
[2025-01-09 23:40:35] INFO | root: Generating text
[2025-01-09 23:40:36] DEBUG | Response: [ToolCallResponse(name='final_answer', args='{"time": "morning", "greeting": "Good morning", "language": "English"}', call_id='c3FME0L7TUGxrxqpN78g8h38')]
[2025-01-09 23:40:36] INFO | Running tool: final_answer
[2025-01-09 

In [70]:
Greeting.model_validate_json(response.contents[0].text)

Greeting(language=<Language.english: 'English'>, greeting='Good morning', time=<GreetingTime.morning: 'morning'>)

For Claude

In [71]:
agent = Agent(
    client=anthropic_client,
    model="claude-3-5-sonnet-20240620",
    max_tokens=500,
    response_schema_as_tool=Greeting,
)

In [72]:
response = agent.generate_text(prompt=prompt)

[2025-01-09 23:40:36] DEBUG | Prompt: [TextPrompt(text='Tell me how to greet in a language. Please pick one language from Japanese, English, French, and German, then\ntell me the greeting message and the time of the day to use it.\n')]
[2025-01-09 23:40:36] INFO | root: Generating text
[2025-01-09 23:40:39] DEBUG | Response: [TextResponse(text="Certainly! I'd be happy to provide you with a greeting in one of the languages you mentioned. Let me choose a language and provide you with a greeting along with the appropriate time of day to use it.\n\nI'll select French for this example."), ToolCallResponse(name='final_answer', args='{"language": "French", "greeting": "Bonjour!", "time": "morning"}', call_id='toolu_0176BXaJRXQZg3WnumE3nAph')]
[2025-01-09 23:40:39] INFO | Running tool: final_answer
[2025-01-09 23:40:39] INFO | Tool: final_answer successfully ran.
[2025-01-09 23:40:39] DEBUG | Final result: [TextResponse(text='{"language": "French", "greeting": "Bonjour!", "time": "morning"}')]

In [73]:
Greeting.model_validate_json(response.contents[0].text)

Greeting(language=<Language.french: 'French'>, greeting='Bonjour!', time=<GreetingTime.morning: 'morning'>)

# Response schema natively supported by each provider

Except a few arguments, Agent module accepts almost all parameters natively supported by each provider as it is. For example, if you want to use native response schema for OpenAI API or Gemini, you can specify that parameter supported by each API.

For OpenAI

In [74]:
prompt = """Tell me how to greet in a language. Please pick one language from Japanese, English, French, and German, then
tell me the greeting message and the time of the day to use it.
"""

agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    response_format=Greeting,  # native parameter for structured output
)

response = agent.generate_text(prompt=prompt)

[2025-01-09 23:40:39] DEBUG | Prompt: [TextPrompt(text='Tell me how to greet in a language. Please pick one language from Japanese, English, French, and German, then\ntell me the greeting message and the time of the day to use it.\n')]
[2025-01-09 23:40:39] INFO | root: Generating text
[2025-01-09 23:40:40] DEBUG | Response: [TextResponse(text='{"language":"Japanese","greeting":"おはようございます (Ohayou gozaimasu)","time":"morning"}')]


In [75]:
Greeting.model_validate_json(response.contents[0].text)

Greeting(language=<Language.japanese: 'Japanese'>, greeting='おはようございます (Ohayou gozaimasu)', time=<GreetingTime.morning: 'morning'>)

For Gemini

In [76]:
# Gemini can't support respones schema includes $defs and $ref so we need to flatten the schema.
# Also parameter type must be upper case.

from langrila.google.gemini_utils import to_gemini_schema

agent = Agent(
    client=google_dev_client,
    model="gemini-2.0-flash-exp",
    response_mime_type="application/json",
    response_schema=to_gemini_schema(Greeting),  # native parameter for structured output
)

response = agent.generate_text(prompt=prompt)

[2025-01-09 23:40:40] DEBUG | Prompt: [TextPrompt(text='Tell me how to greet in a language. Please pick one language from Japanese, English, French, and German, then\ntell me the greeting message and the time of the day to use it.\n')]
[2025-01-09 23:40:40] INFO | root: Generating text
[2025-01-09 23:40:42] DEBUG | Response: [TextResponse(text='{\n  "greeting": "こんにちは",\n  "language": "Japanese",\n  "time": "afternoon"\n}')]


In [77]:
Greeting.model_validate_json(response.contents[0].text)

Greeting(language=<Language.japanese: 'Japanese'>, greeting='こんにちは', time=<GreetingTime.afternoon: 'afternoon'>)

Please note that different provider has the different specification and limitation of the native response schema.

# Customize internal prompt

Some agent framework uses internal prompt that is not customizable. This might affect the agent behaviour using another language like Japanese. Agent in langrila has internal prompts as well, for example, that is used for validation error retry, but that is customizable.

In [78]:
from langrila import InternalPrompt

internal_prompt = InternalPrompt()

All the internal prompts are:

In [79]:
internal_prompt.model_dump()

{'error_retry': 'Please fix the error on tool use. If the validation error is raised, reflect the conversation history and try to find the correct answer. If there is no fact to answer the user, try to run other tools to get necessary information. ',
 'no_tool_use_retry': "Decide the next action based on the conversation. If you have all information for answering the user, run 'final_result' tool. If you need more information, invoke other tool to get necessary information. ",
 'planning': 'Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\n{user_input}\n\nCapabilities:\n{capabilities}',
 'do_plan': 'Put the plan into action.'}

If you want to customize internal prompt, you can configure it via `AgentConfig`.

In [80]:
from langrila import AgentConfig

agent_config = AgentConfig(
    internal_prompt=InternalPrompt(
        error_retry="エラーを修正してください。",  # replace retry prompt with Japanese
    ),
)

agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    agent_config=agent_config,
)

# Appendix

Langrila supports o1-family for OpenAI API. (Precisely, langrila doesn't depend on the model name.)

In [81]:
agent = Agent(
    client=openai_client,
    model="o1-mini-2024-09-12",
)

In [82]:
response = agent.generate_text(prompt=prompt1)
response

[2025-01-09 23:40:42] DEBUG | Prompt: [TextPrompt(text='Hello. How are you today?')]
[2025-01-09 23:40:42] INFO | root: Generating text
[2025-01-09 23:40:44] DEBUG | Response: [TextResponse(text="Hello! I'm doing well, thank you for asking. How can I help you today?")]


Response(type='Response', role='assistant', contents=[TextResponse(text="Hello! I'm doing well, thank you for asking. How can I help you today?")], usage=<langrila.core.usage.NamedUsage object at 0x7fddcf612bf0>, raw=ChatCompletion(id='chatcmpl-Ano0Be0NHMDWUR4qNRtmgkmNVT31S', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Hello! I'm doing well, thank you for asking. How can I help you today?", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1736433643, model='o1-mini-2024-09-12', object='chat.completion', service_tier=None, system_fingerprint='fp_f56e40de61', usage=CompletionUsage(completion_tokens=158, prompt_tokens=14, total_tokens=172, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=128, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))), name=None, is_last_chunk=Non

In [83]:
response.usage.items()

dict_items([('root', Usage(model_name='o1-mini-2024-09-12', prompt_tokens=14, output_tokens=158))])